# Tracking Test Usage and Costs

Because we use OpenAI, running tests isn't free. Each test makes API calls that cost money.

It would be nice to know how much money we actually spend on running our test cases.

One thing you should do is to create a separate key for tests only. This way you can easily monitor how much you spend on tests in general.

# Usage Tracking Strategy

We can get usage information from `run_results.usage()` and collect all the usages from all test runs to see the total costs.

An alternative would be monkey-patching the run method from Pydantic AI's Agent class, so during tests all usages are automatically collected. (You can do the same with any other framework you use.)

At the end, when we finish running all the tests, we can use it to generate a comprehensive cost report.

Let's create a file `patch_agent.py` (`test/patch_agent.py`):

In [ ]:
from functools import wraps
import inspect
from typing import Any

# Don't import pydantic_ai types here

# Central in-process collector
_USAGE_RECORDS = {}


def _record_usage_from_result(agent, result):
    try:
        model_name = agent.model.model_name
        usage = result.usage()

        if usage is None:
            return

        if model_name not in _USAGE_RECORDS:
            _USAGE_RECORDS[model_name] = []
        _USAGE_RECORDS[model_name].append(usage)
    except Exception:
        return


def _wrap_run_callable(orig_run):
    """Return an async wrapper which calls orig_run and records usage."""

    @wraps(orig_run)
    async def wrapped(self, *args, **kwargs):
        result = await orig_run(self, *args, **kwargs)
        try:
            _record_usage_from_result(self, result)
        except Exception:
            pass
        return result

    return wrapped


def install_usage_collector() -> bool:
    installed = False
    
    from pydantic_ai import Agent as PAAgent  # type: ignore

    try:
        if not hasattr(PAAgent, "_orig_run_for_tests"):
            orig_run = getattr(PAAgent, "run", None)
            setattr(PAAgent, "_orig_run_for_tests", orig_run)
            if orig_run is not None and inspect.iscoroutinefunction(orig_run):
                PAAgent.run = _wrap_run_callable(orig_run)
            installed = True
    except Exception:
        pass

    return installed


def get_usage_aggregated():
    from pydantic_ai import RunUsage  # type: ignore

    aggregated = {}

    for model_name, usage_list in _USAGE_RECORDS.items():
        total = RunUsage()

        for usage in usage_list:
            total.incr(usage)

        aggregated[model_name] = total

    return aggregated


def print_report_usage() -> Any:
    from toyaikit.pricing import PricingConfig
    pricing = PricingConfig()
    print('\n=== USAGE REPORT ===')

    aggregated = get_usage_aggregated()

    for model_name, usage in aggregated.items():
        cost = pricing.calculate_cost(
            model=model_name,
            input_tokens=usage.input_tokens,
            output_tokens=usage.output_tokens
        )
        print(f'Model: {model_name}, Cost: {cost}')

    print('====================\n')

Here is what's happening:

- _USAGE_RECORDS is a global dictionary that stores usage data for each model.
- _record_usage_from_result() extracts usage information from an agent result and stores it in our global collector.
- _wrap_run_callable() creates a wrapper around the original agent run method. It calls the original run method and then records the usage data.
- install_usage_collector() monkey-patches the Pydantic AI Agent class. It replaces the original run method with our wrapped version that collects usage data.
- get_usage_aggregated() combines all individual usage records for each model into totals.
- print_report_usage() generates and displays a cost report using ToyAIKit's pricing information. It shows the total cost for each model used during testing.

We need are using ToyAIKit for calculating the prices

In [ ]:
!uv add toyaikit

# Integration with Test Suite

Create `conftest.py` (`tests/conftest.py`)

In [ ]:
from tests import patch_agent
patch_agent.install_usage_collector()

It's very important to add this at the top. We want to make sure we install the usage collector before the first instance of Agent is created. Otherwise our monkey-patching won't work properly.

Finally, we show the cost report when the test session finishes:

In [ ]:
def pytest_sessionfinish(session, exitstatus):
    from tests import patch_agent
    patch_agent.print_report_usage()


This pytest hook runs automatically after all tests complete. It provides a summary of total API costs incurred during the test session, helping you monitor and budget for testing expenses.

You can also save these results somewhere, so later you can analyze the costs.

That's how we can monitor costs of a single test set run.

So the complete `tests/conftest.py` looks like this:

In [ ]:
from tests import patch_agent
patch_agent.install_usage_collector()

def pytest_sessionfinish(session, exitstatus):
    from tests import patch_agent
    patch_agent.print_report_usage()